In [ ]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install nlpaug

     |████████████████████████████████| 389kB 18.0MB/s 


In [ ]:
!pip install tqdm  >> /dev/null

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2021-03-01 05:41:37--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-03-01 05:41:37--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-03-01 05:41:37--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
 nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix,classification_report

In [ ]:
from tqdm import tqdm
from sklearn.utils import shuffle

In [ ]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as naf

from nlpaug.util import Action

In [ ]:
header_list = ["text","intent","buffer"]
train = pd.read_csv("/content/english_hope_train.csv", '\t',header=None,names=header_list)
valid = pd.read_csv("/content/english_hope_dev.csv", '\t',header=None,names=header_list)

In [ ]:
train = train.drop(labels='buffer',axis=1)
valid = valid.drop(labels='buffer',axis=1)

In [ ]:
train = train.loc[train['intent']!='not-English']
valid = valid.loc[valid['intent']!='not-English']

In [ ]:
print(train.shape, valid.shape)

(22740, 2) (2841, 2)


In [ ]:
np.unique(train['intent'])

array(['Hope_speech', 'Non_hope_speech'], dtype=object)

In [ ]:
train[train['intent']=='Hope_speech'].shape

(1962, 2)

In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot",
                   "can't've": "cannot have", "'cause": "because", "could've": "could have",
                   "couldn't": "could not", "couldn't've": "could not have","didn't": "did not",
                   "doesn't": "does not", "don't": "do not", "hadn't": "had not",
                   "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not",
                   "he'd": "he would", "he'd've": "he would have", "he'll": "he will",
                   "he'll've": "he will have", "he's": "he is", "how'd": "how did",
                   "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                   "I'd": "I would", "I'd've": "I would have", "I'll": "I will",
                   "I'll've": "I will have","I'm": "I am", "I've": "I have",
                   "i'd": "i would", "i'd've": "i would have", "i'll": "i will",
                   "i'll've": "i will have","i'm": "i am", "i've": "i have",
                   "isn't": "is not", "it'd": "it would", "it'd've": "it would have",
                   "it'll": "it will", "it'll've": "it will have","it's": "it is",
                   "let's": "let us", "ma'am": "madam", "mayn't": "may not",
                   "might've": "might have","mightn't": "might not","mightn't've": "might not have",
                   "must've": "must have", "mustn't": "must not", "mustn't've": "must not have",
                   "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                   "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                   "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would",
                   "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have",
                   "she's": "she is", "should've": "should have", "shouldn't": "should not",
                   "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                   "this's": "this is",
                   "that'd": "that would", "that'd've": "that would have","that's": "that is",
                   "there'd": "there would", "there'd've": "there would have","there's": "there is",
                       "here's": "here is",
                   "they'd": "they would", "they'd've": "they would have", "they'll": "they will",
                   "they'll've": "they will have", "they're": "they are", "they've": "they have",
                   "to've": "to have", "wasn't": "was not", "we'd": "we would",
                   "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                   "we're": "we are", "we've": "we have", "weren't": "were not",
                   "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                   "what's": "what is", "what've": "what have", "when's": "when is",
                   "when've": "when have", "where'd": "where did", "where's": "where is",
                   "where've": "where have", "who'll": "who will", "who'll've": "who will have",
                   "who's": "who is", "who've": "who have", "why's": "why is",
                   "why've": "why have", "will've": "will have", "won't": "will not",
                   "won't've": "will not have", "would've": "would have", "wouldn't": "would not",
                   "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                   "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                   "you'd": "you would", "you'd've": "you would have", "you'll": "you will",
                   "you'll've": "you will have", "you're": "you are", "you've": "you have" }

In [ ]:
pip install Unidecode

     |████████████████████████████████| 245kB 8.4MB/s 


In [ ]:
import codecs
import unidecode
import re
import spacy
nlp = spacy.load('en')

def spacy_cleaner(text):
    try:
        decoded = unidecode.unidecode(codecs.decode(text, 'unicode_escape'))
    except:
        decoded = unidecode.unidecode(text)
    apostrophe_handled = re.sub("’", "'", decoded)
    expanded = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in apostrophe_handled.split(" ")])
    parsed = nlp(expanded)
    final_tokens = []
    for t in parsed:
        if t.is_punct or t.is_space or t.like_num or t.like_url or str(t).startswith('@'):
            pass
        else:
            if t.lemma_ == '-PRON-':
                final_tokens.append(str(t))
            else:
                sc_removed = re.sub("[^a-zA-Z]", '', str(t.lemma_))
                if len(sc_removed) > 1:
                    final_tokens.append(sc_removed)
    joined = ' '.join(final_tokens)
    spell_corrected = re.sub(r'(.)\1+', r'\1\1', joined)
    return spell_corrected

In [ ]:
train['text'] = [spacy_cleaner(t) for t in train.text]

In [ ]:
valid['text'] = [spacy_cleaner(t) for t in valid.text]

In [ ]:
valid.head()

text           intent
0             that it like I do not like that statue  Non_hope_speech
1  Counting money that she be give for promote hy...  Non_hope_speech
2   Hernandez never say to be intolerant and hateful  Non_hope_speech
3         wow an opinion be that I do not like color  Non_hope_speech
4  WOWThat be so so inspiring and incredible this...      Hope_speech

In [ ]:
np.random.seed(500)

In [ ]:
train['text']

0        these tiktok radiate gay chaotic energy and lo...
1                    again He get kill for use false money
2                    It be not that all life do not matter
3        be it really that difficult to understand blac...
4        whenever we say black be not that racist why d...
                               ...                        
22757    It be load of bollock every life matter simple...
22758    no say it because all live matter deku would n...
22759                              God say her life matter
22760    this video be just shit bunch of whiny ass bit...
22761     Mc Fortnut she do month ago in west church video
Name: text, Length: 22740, dtype: object

In [ ]:
# model_type: word2vec, glove or fasttext
aug_w2v = naw.WordEmbsAug(
#     model_type='word2vec', model_path='../input/nlpword2vecembeddingspretrained/GoogleNews-vectors-negative300.bin',
    model_type='glove', model_path='/content/glove.6B.100d.txt',
    action="substitute")

In [ ]:
#Code for increasing data samples
#Run if you want to increase the data samples
def augment_text(df,samples=10000,pr=0.2):
    aug_w2v.aug_p=pr
    new_text=[]

    ##selecting the minority class samples
    df_n=df[df.intent=='Hope_speech'].reset_index(drop=True)

    ## data augmentation loop
    for i in tqdm(np.random.randint(0,len(df_n),samples)):

            text = df_n.iloc[i]['text']
            augmented_text = aug_w2v.augment(text)
            new_text.append(augmented_text)


    ## dataframe
    new=pd.DataFrame({'text':new_text,'intent':'Hope_speech'})
    df=shuffle(df.append(new).reset_index(drop=True))
    return df,new

train_preprocessed, new_dp = augment_text(train_preprocessed)

100%|██████████| 10000/10000 [31:49<00:00,  5.24it/s]


In [ ]:
train.dropna(subset=['text'], inplace=True)

In [ ]:
# Step - a : Remove blank rows if any.
#train['text'].dropna(inplace=True)
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
train['text'] = [entry.lower() for entry in train['text']]
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
train['text']= [word_tokenize(entry) for entry in train['text']]
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(train['text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    train.loc[index,'text_final'] = str(Final_words)

In [ ]:
# Step - a : Remove blank rows if any.
#valid['text'].dropna(inplace=True)
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
valid['text'] = [entry.lower() for entry in valid['text']]
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
valid['text']= [word_tokenize(entry) for entry in valid['text']]
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(valid['text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    valid.loc[index,'text_final'] = str(Final_words)

In [ ]:
train.dropna(subset=['intent'], inplace=True)

In [ ]:
train.dropna(subset=['text_final'], inplace=True)

In [ ]:
train.shape

(32728, 3)

In [ ]:
valid.dropna(subset=['intent'], inplace=True)

In [ ]:
valid.dropna(subset=['text_final'], inplace=True)

In [ ]:
valid.shape

(2841, 3)

In [ ]:
train.head()

text  ...                                         text_final
0  [these, tiktok, radiate, gay, chaotic, energy,...  ...  ['tiktok', 'radiate', 'gay', 'chaotic', 'energ...
1     [again, he, get, kill, for, use, false, money]  ...           ['get', 'kill', 'use', 'false', 'money']
2    [it, be, not, that, all, life, do, not, matter]  ...                                 ['life', 'matter']
3  [be, it, really, that, difficult, to, understa...  ...  ['really', 'difficult', 'understand', 'black',...
4  [whenever, we, say, black, be, not, that, raci...  ...  ['whenever', 'say', 'black', 'racist', 'say', ...

[5 rows x 3 columns]

In [ ]:
valid.head()

text  ...                                         text_final
0   [that, it, like, i, do, not, like, that, statue]  ...                         ['like', 'like', 'statue']
1  [counting, money, that, she, be, give, for, pr...  ...  ['count', 'money', 'give', 'promote', 'hysteri...
2  [hernandez, never, say, to, be, intolerant, an...  ...  ['hernandez', 'never', 'say', 'intolerant', 'h...
3  [wow, an, opinion, be, that, i, do, not, like,...  ...                ['wow', 'opinion', 'like', 'color']
4  [wowthat, be, so, so, inspiring, and, incredib...  ...  ['wowthat', 'inspiring', 'incredible', 'speech...

[5 rows x 3 columns]

In [ ]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(train['text_final'],train['intent'],test_size=0.3)

In [ ]:
Valid_X = valid['text_final']
Valid_Y = valid['intent']

In [ ]:
#Train_Y=pd.Series.to_numpy(Train_Y)


In [ ]:
train['intent'].unique()

array(['Hope_speech', 'Non_hope_speech'], dtype=object)

In [ ]:
Train_Y.unique()

array(['Non_hope_speech', 'Hope_speech'], dtype=object)

In [ ]:
Train_Y = Train_Y.replace(to_replace ="Non_hope_speech", value =0)
Train_Y = Train_Y.replace(to_replace ="Hope_speech", value =1)
Train_Y.unique()

array([0, 1])

In [ ]:
Test_Y = Test_Y.replace(to_replace ="Non_hope_speech", value =0)
Test_Y = Test_Y.replace(to_replace ="Hope_speech", value =1)
Train_Y.unique()

array([0, 1])

In [ ]:
Valid_Y = Valid_Y.replace(to_replace ="Non_hope_speech", value =0)
Valid_Y = Valid_Y.replace(to_replace ="Hope_speech", value =1)
Valid_Y.unique()

array([0, 1])

In [ ]:
type(Valid_Y[0])

numpy.int64

In [ ]:
# Other method for doing operations done in above 3 cells
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [ ]:
Valid_Y = Encoder.fit_transform(Valid_Y)

In [ ]:
#Tokenization
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(train['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [ ]:
Valid_X_Tfidf = Tfidf_vect.transform(Valid_X)

In [ ]:
print(Tfidf_vect.vocabulary_)

{'radiate': 3617, 'gay': 1835, 'chaotic': 742, 'energy': 1478, 'love': 2671, 'get': 1861, 'kill': 2485, 'use': 4702, 'false': 1628, 'money': 2909, 'life': 2602, 'matter': 2766, 'really': 3656, 'difficult': 1248, 'understand': 4656, 'black': 497, 'live': 2627, 'exclusive': 1575, 'true': 4598, 'general': 1844, 'statement': 4230, 'white': 4851, 'asian': 290, 'context': 954, 'simply': 4068, 'name': 2975, 'movement': 2938, 'high': 2069, 'rate': 3639, 'police': 3419, 'violence': 4751, 'people': 3317, 'refer': 3675, 'behave': 437, 'like': 2608, 'treat': 4577, 'example': 1567, 'whenever': 4842, 'say': 3891, 'racist': 3615, 'american': 171, 'bos': 557, 'donat': 1337, 'know': 2504, 'shea': 4009, 'actually': 61, 'lgbtq': 2591, 'tho': 4472, 'funny': 1810, 'end': 1474, 'larry': 2529, 'argue': 256, 'haha': 1969, 'save': 3885, 'music': 2961, 'lot': 2664, 'army': 263, 'unit': 4672, 'network': 3027, 'engineer': 1483, 'currently': 1075, 'work': 4912, 'teach': 4402, 'man': 2712, 'woman': 4903, 'look': 26

In [ ]:
print(Train_X_Tfidf)

  (0, 4532)	0.3898600572662303
  (0, 2161)	0.29575697178000493
  (0, 1987)	0.3664409073271041
  (0, 1302)	0.4887554299339212
  (0, 1082)	0.3740785935503778
  (0, 965)	0.35443627753749624
  (0, 893)	0.3490218783285998
  (1, 4958)	0.23724027236950224
  (1, 4334)	0.35754857890918157
  (1, 2608)	0.1883042146400151
  (1, 1703)	0.28699094937179914
  (1, 1566)	0.49832283412282224
  (1, 1562)	0.4057457577263028
  (1, 1352)	0.38066041846919013
  (1, 956)	0.37442778142562744
  (2, 4980)	0.445376888452522
  (2, 4784)	0.31124070317396046
  (2, 3467)	0.5535151388506615
  (2, 2704)	0.30119995704061675
  (2, 2482)	0.3492147068597451
  (2, 1060)	0.4309493211212361
  (3, 4927)	0.10809612098711059
  (3, 4903)	0.12557623564833348
  (3, 4886)	0.20853850913510927
  (3, 4745)	0.12819885806167414
  :	:
  (15897, 1154)	0.593575841022778
  (15897, 880)	0.5723648021882091
  (15897, 442)	0.40823412208859516
  (15898, 4903)	0.39888193267831235
  (15898, 4836)	0.5384118639498073
  (15898, 4784)	0.356871925141921
 

In [ ]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score on valid set-> ",accuracy_score(predictions_NB, Test_Y)*100)

In [ ]:
# accuracy on test set
predictions_NB_T = Naive.predict(Valid_X_Tfidf)
print("Naive Bayes Accuracy Score on test set-> ",accuracy_score(predictions_NB_T, Valid_Y)*100)

Naive Bayes Accuracy Score ->  90.45438534695315


In [ ]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score on valid set-> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  91.27053990610328


In [ ]:
#accuracy on test set
predictions_SVM_T = SVM.predict(Valid_X_Tfidf)
print("SVM Accuracy Score test set -> ",accuracy_score(predictions_SVM_T, Valid_Y)*100)

In [ ]:
print ('\n*Classification Report: Naive bayes\n', classification_report(Valid_Y, predictions_NB_T))
confusion_matrix(Valid_Y, predictions_NB_T)


*Classification Report: Naive bayes
               precision    recall  f1-score   support

           0       0.90      1.00      0.95      2568
           1       0.00      0.00      0.00       271

    accuracy                           0.90      2839
   macro avg       0.45      0.50      0.47      2839
weighted avg       0.82      0.90      0.86      2839



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[2568,    0],
       [ 271,    0]])

In [ ]:
# For Naive bayes
tn, fp, fn, tp = confusion_matrix(Valid_Y, predictions_NB_T).ravel()
print(tn,fp,fn,tp)

2568 0 271 0


In [ ]:
# Training set accuracy SVM
predictions_SVM_Tr = SVM.predict(Train_X_Tfidf)
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM_Tr, Train_Y)*100)

SVM Accuracy Score ->  91.4161740661552


In [ ]:
#Training set accuracy Naive bayes
predictions_NB_Tr = Naive.predict(Train_X_Tfidf)
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB_Tr, Train_Y)*100)

Naive Bayes Accuracy Score ->  91.42246258332285


In [ ]:
print ('\n*Classification Report: Naive bayes\n', classification_report(Valid_Y, predictions_SVM_T))
confusion_matrix(Valid_Y, predictions_SVM_T)


*Classification Report: Naive bayes
               precision    recall  f1-score   support

           0       0.90      1.00      0.95      2568
           1       0.00      0.00      0.00       271

    accuracy                           0.90      2839
   macro avg       0.45      0.50      0.47      2839
weighted avg       0.82      0.90      0.86      2839



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[2568,    0],
       [ 271,    0]])

In [ ]:
#For SVM
tn, fp, fn, tp = confusion_matrix(Valid_Y, predictions_SVM_T).ravel()
print(tn,fp,fn,tp)

2568 0 271 0
